## Programa Presidencial “Bono Vida Mejor" en sus Dominios Rural y Urbano
### Ejercicio de transferencias

In [1]:
# Libraries 
import os
import random
import numpy as np
import pandas as pd
from dotenv import load_dotenv



In [2]:
# Working directory
load_dotenv('/home/ec2-user/SageMaker/.env')
sclbucket = os.getenv('sclbucket')
scldatalake = 's3://{0}'.format(sclbucket)

In [5]:
%%capture
# Import data 
recurso = 'Household Socio-Economic Surveys/Harmonized Household Surveys/concat'
file    = 'harmonized-latest-v3.csv'
data    = pd.read_csv(f'{scldatalake}/{recurso}/{file}')

In [6]:
# Data Honduras
data_hnd = data[data.pais_c == "HND"]

### Pobreza nacional

In [12]:
# Pobreza nacional
poor_national   = data_hnd.groupby("poor_national").factor_ci.sum()  / data_hnd.groupby("poor_national").factor_ci.sum().sum()
poor_national   = poor_national[1]

# Pobreza extrema nacional
poor_e_national = data_hnd.groupby("poor_e_national").factor_ci.sum() / data_hnd.groupby("poor_national").factor_ci.sum().sum()
poor_e_national = poor_e_national[1]

# Results
print("Pobreza nacional        : {:.2f}%".format(poor_national  *100))
print("Pobreza extrema nacional: {:.2f}%".format(poor_e_national*100))

Pobreza nacional        : 63.03%
Pobreza extrema nacional: 40.62%


### Simulador

In [198]:
hh.groupby("poor_national").factor_ci.sum()  / hh.groupby("poor_national").factor_ci.sum().sum()

poor_national
0.0    0.369657
1.0    0.630343
Name: factor_ci, dtype: float64

In [199]:
result = simulacion(0, 0, [0], 100, 1)

In [200]:
result.groupby("poor_national_new").factor_ci.sum()  / result.groupby("poor_national_new").factor_ci.sum().sum()

poor_national_new
0.0    0.369657
1.0    0.630343
Name: factor_ci, dtype: float64

In [219]:
result = simulacion(350000, 0.5, [335,200], 16, 1)

In [218]:
result.groupby("poor_national_new").factor_ci.sum()  / result.groupby("poor_national_new").factor_ci.sum().sum()

poor_national_new
0.0    0.375331
1.0    0.624669
Name: factor_ci, dtype: float64

In [221]:
def simulacion(nbenef, pbenef_rural, monto, max_edad, pob_grupo = 1):
    # Seed
    random.seed(1)
    
    # Definir población elegible
    # Copiar data
    hh = data_hnd.copy()
        
    # ID beneficiarios
    hh['id_ben']       = 0
    hh['id_ben_rural'] = 0
    hh['id_ben_urban'] = 0
        
    # Indetificar hogares con niños y adolescentes 
    hh['id_max_edad'] = np.where((hh.miembros_ci == 1) & (hh.edad_ci <= max_edad),1,0)
    hh['id_max_edad'] = hh.groupby('idh_ch').id_max_edad.transform(max)
        
    # Identificar poblacion en pobreza y hogares con niños y adolescentes
    if pob_grupo == 1:
        hh_temp = hh.loc[(hh.poor_national   == 1) & (hh.id_max_edad == 1)]
        hh_temp = hh_temp.sample(frac = 1).iterrows()
    else: 
        hh_temp = hh.loc[(hh.poor_e_national == 1) & (hh.id_max_edad == 1)]
        hh_temp = hh_temp.sample(frac = 1).iterrows()
    
    # Identificar beneficiarios y asignar monto
    if pbenef_rural > 0:
        nbenef_rural = nbenef * pbenef_rural
        nbenef_urban = nbenef * (1 - pbenef_rural)
    
        # Seleccionar aleatoriamente a `n` individuos 
        iter_r = 0 
        iter_u = 0
        for i in hh_temp:
            if i[1]['zona_c'] == 0:
                if (iter_r +  i[1]['factor_ch']) <= nbenef_rural:
                    iter_r += i[1]['factor_ch'] 
                    hh.loc[(hh.idh_ch == i[1]['idh_ch']) & (hh.idp_ci == i[1]['idp_ci']),'id_ben_rural'] = 1
                if (iter_r +  i[1]['factor_ch']) >  nbenef_rural:
                    break
                
            else: 
                if (iter_u +  i[1]['factor_ch']) <= nbenef_urban:
                    iter_u += i[1]['factor_ch'] 
                    hh.loc[(hh.idh_ch == i[1]['idh_ch']) & (hh.idp_ci == i[1]['idp_ci']),'id_ben_urban'] = 1
                if (iter_u +  i[1]['factor_ch']) >  nbenef_urban:
                    break
                
        # Asignar transferencia
        var_inc     = 'pc_ytot_ch_ofi_new'
        hh[var_inc] = np.where(hh.id_ben_rural == 1, monto[0] / hh.nmiembros_ch, 0) + hh.pc_ytot_ch_ofi
        hh[var_inc] = np.where(hh.id_ben_urban == 1, monto[1] / hh.nmiembros_ch, 0) + hh.pc_ytot_ch_ofi_new
        
        # Identificar numero de individuos en pobreza 
        if pob_grupo == 1:
            var_pov     = 'poor_national_new'
            hh[var_pov] = (hh[var_inc] < hh.lp_ci).astype(int)
            hh.loc[hh.pc_ytot_ch_ofi.isna(),var_pov] = np.nan
        else:
            var_pov     = 'poor_e_national_new'
            hh[var_pov] = (hh[var_inc] < hh.lpe_ci).astype(int)
            hh.loc[hh.pc_ytot_ch_ofi.isna(),var_pov] = np.nan
                    
    else:
        # Seleccionar aleatoriamente a `n` individuos 
        iter_ = 0 
        for i in hh_temp:
            if (iter_ +  i[1]['factor_ch']) <= nbenef:
                iter_ += i[1]['factor_ch']
                hh.loc[(hh.idh_ch == i[1]['idh_ch']) & (hh.idp_ci == i[1]['idp_ci']),'id_ben'] = 1
            if (iter_ +  i[1]['factor_ch']) >  nbenef:
                break  
    
        # Asignar transferencia
        var_inc     = 'pc_ytot_ch_ofi_new'
        hh[var_inc] = np.where(hh.id_ben == 1, monto[0] / hh.nmiembros_ch, 0) + hh.pc_ytot_ch_ofi
    
        # Identificar numero de individuos en pobreza
        if pob_grupo == 1:
            var_pov     = 'poor_national_new'
            hh[var_pov] = (hh[var_inc] < hh.lp_ci).astype(int)
            hh.loc[hh.pc_ytot_ch_ofi.isna(),var_pov] = np.nan
        else:
            var_pov     = 'poor_e_national_new'
            hh[var_pov] = (hh[var_inc] < hh.lpe_ci).astype(int)
            hh.loc[hh.pc_ytot_ch_ofi.isna(),var_pov] = np.nan
    
    return hh 

### Resultados